<a href="https://colab.research.google.com/github/riozaky/MachineLearning/blob/main/week9/concisce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%matplotlib inline
import random
import tensorflow as tf
from d2l import tensorflow as d2l

In [4]:
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# **3.3.2. Reading the Dataset**
Rather than rolling our own iterator, we can call upon the existing API in a framework to read data. We pass in features and labels as arguments and specify batch_size when instantiating a data iterator object. Besides, the boolean value is_train indicates whether or not we want the data iterator object to shuffle the data on each epoch (pass through the dataset).

In [6]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a TensorFlow data iterator."""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [7]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.38937184,  1.3867269 ],
        [-0.76338434, -0.07603722],
        [-0.7963677 , -1.1608542 ],
        [ 0.21869661,  0.4278761 ],
        [-1.9957068 , -0.09048171],
        [-0.88988626,  0.6324615 ],
        [ 0.43412125,  0.934523  ],
        [-1.24614   ,  1.4439404 ],
        [ 0.02665309,  0.01030167],
        [ 0.20256123,  0.31200227]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[-1.2998898 ],
        [ 2.9137485 ],
        [ 6.5644946 ],
        [ 3.1897588 ],
        [ 0.52814114],
        [ 0.25662687],
        [ 1.8664755 ],
        [-3.1982794 ],
        [ 4.219053  ],
        [ 3.5397167 ]], dtype=float32)>)

# **3.3.3. Defining the Model**
When we implemented linear regression from scratch in Section 3.2, we defined our model parameters explicitly and coded up the calculations to produce output using basic linear algebra operations. You should know how to do this. But once your models get more complex, and once you have to do this nearly every day, you will be glad for the assistance. The situation is similar to coding up your own blog from scratch. Doing it once or twice is rewarding and instructive, but you would be a lousy web developer if every time you needed a blog you spent a month reinventing the wheel.

In [8]:
# `keras` is the high-level API for TensorFlow
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1))

# **3.3.4. Initializing Model Parameters**
Before using net, we need to initialize the model parameters, such as the weights and bias in the linear regression model. Deep learning frameworks often have a predefined way to initialize the parameters. Here we specify that each weight parameter should be randomly sampled from a normal distribution with mean 0 and standard deviation 0.01. The bias parameter will be initialized to zero

In [9]:
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

# **3.3.5. Defining the Loss Function**

In [10]:
loss = tf.keras.losses.MeanSquaredError()

# **3.3.6. Defining the Optimization Algorithm**

In [11]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

# **3.3.7. Training**
You might have noticed that expressing our model through high-level APIs of a deep learning framework requires comparatively few lines of code. We did not have to individually allocate parameters, define our loss function, or implement minibatch stochastic gradient descent. Once we start working with much more complex models, advantages of high-level APIs will grow considerably. However, once we have all the basic pieces in place, the training loop itself is strikingly similar to what we did when implementing everything from scratch.

In [12]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000244
epoch 2, loss 0.000091
epoch 3, loss 0.000090


In [13]:
w = net.get_weights()[0]
print('error in estimating w', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('error in estimating b', true_b - b)

error in estimating w tf.Tensor([ 0.0003947  -0.00038433], shape=(2,), dtype=float32)
error in estimating b [-0.00062847]
